[View in Colaboratory](https://colab.research.google.com/github/sapjunior/chulacv2018/blob/master/Lab%206%20-%20Convolutional%20Neural%20Network.ipynb)

In [0]:
## This Block is for Google Colaboratory user. Don't forget to enable GPU in Runtime->Change Runtime Type-> Hardware Accelerator -> GPU
## Install missing packages for Google Colaboratory ##
!pip3 install http://download.pytorch.org/whl/cu92/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
!pip3 install kaggle

## Load Library

In [0]:
import numpy as np
import cv2
import urllib
import tarfile
from matplotlib import pyplot as plt
from tqdm import tqdm
from PIL import Image
import numpy as np
import csv
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

## Setup Kaggle API KEY

In [0]:
# On the kaggle website, click on the profile image on upper right section and select "My Account"
# Scroll down to the API section and click on "Create New API Token"
# Then, open the downloaded file, you will found your api key there.

api_token = {"username":"kaggleUSERNAME","key":"kaggleAPIKEY"}

import json
import zipfile
import os
!mkdir /content
with open('/content/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /content/kaggle.json
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle

## Simple K-Nearest Neighbour on CIFAR-10 dataset
---

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. 

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class. 

![alt text](https://kaggle2.blob.core.windows.net/competitions/kaggle/3649/media/cifar-10.png)

In [0]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
  
# Download CIFAR-10 dataset from https://www.cs.toronto.edu/~kriz/cifar.html
urllib.request.urlretrieve ('https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz', 'cifar-10-python.tar.gz')
with tarfile.open('cifar-10-python.tar.gz') as tar:
    tar.extractall()
    
# Show files in extract directory
!ls cifar-10-batches-py/
cifarClassName = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [0]:
# For each training batch read and concatenate into one big numpy array
# You may find the format of CIFAR-10 in Data Layout section https://www.cs.toronto.edu/~kriz/cifar.html

# Train Data
cifarImages = np.empty((0,3072), dtype=np.uint8)
cifarLabels = np.empty((0,), dtype=np.uint8)
for batchNo in range(1,6):
  dataDict = unpickle('cifar-10-batches-py/data_batch_'+str(batchNo))
  cifarImages = np.vstack((cifarImages, dataDict[b'data']))
  cifarLabels = np.hstack((cifarLabels, dataDict[b'labels']))
print('Image Array Shape:', cifarImages.shape)
print('Label Array Shape:',cifarLabels.shape)

# Test Data
dataDict = unpickle('cifar-10-batches-py/test_batch')
cifarTestImages = dataDict[b'data']
cifarTestLabels = np.array(dataDict[b'labels'])

In [0]:
# Show samples from CIFAR-10
randIdxs = np.random.permutation(cifarLabels.shape[0])[0:5]
randomImages = np.empty((32,0,3), dtype=np.uint8)
randomImageClasses = []
for randIdx in randIdxs:
  randomImages = np.hstack((randomImages, cifarImages[randIdx,:].reshape(3,32,32).transpose(1,2,0)))
  randomImageClasses.append(cifarClassName[cifarLabels[randIdx]])
plt.imshow(randomImages)
plt.show()
print(randomImageClasses)

Compute the L1 Distance between unknown instance from test image and training images. Given given two images and representing them as vectors $I_1$ and $I_2$, the L1 distance can be calculated by using the following equation.
\begin{equation}d_1(I_1,I_2) = \sum|I_1-I_2| \end{equation}

Pick the k-nearest class as the answer, where k = 3.

In [0]:
testIdx = 2
unknownInstance = cifarTestImages[testIdx,:]
unknownInstanceImage = unknownInstance.reshape(3,32,32).transpose(1,2,0)
print('Ground Truth Class:',cifarClassName[cifarTestLabels[testIdx]])
plt.figure(figsize=(2,2))
plt.imshow(unknownInstanceImage)
plt.show()

### FILL HERE ###


####

In [0]:
## Compute the accuracy on test set ##
## FILL HERE ##

#print('Using L1 Distance Accuracy:',L1DistAccuracy)

L2 Distance between unknown instance from test image and training images. Given given two images and representing them as vectors $I_1$ and $I_2$, the L2 distance can be calculated by using the following equation.
\begin{equation}d_2(I_1,I_2) = \sqrt{\sum(I_1-I_2)^2} \end{equation}

In [0]:
## Try to adjust k and using other distance matrix (may be L2)
## Compare the output accuracy with default setting
## FILL HERE ##


In [0]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision

## Quick PyTorch Tensor Tutorial

In [0]:
# You can use any command you use in numpy with torch Tensor
tempTensor = torch.arange(100,125)
print(tempTensor)
tempTensor = tempTensor.reshape(5,5)

# Reshape
print('Reshape Output:',tempTensor,'Tensor Shape:',tempTensor.shape)
# Tensor Indexing
print(tempTensor[:,1:3])

# Find min,max value
print('Min val:',torch.min(tempTensor),'Max val:',torch.max(tempTensor))
# Find min,max idx
print('Min val idx:',torch.argmin(tempTensor),'Max val idx:',torch.argmax(tempTensor))

# All Ones
allOneTensor = torch.ones((3,3)) * 7
# All Zeros
allZeroTensor = torch.zeros((3,3))
# Concatenate
concatTensor = torch.cat((allOneTensor,allZeroTensor), dim=0)

print(concatTensor,'Tensor Shape:', concatTensor.shape)

In [0]:
# Autograd Feature
x = torch.ones((1), requires_grad=True)*9
print('x:',x)
y = (x**2)+1
print('y:',y)
print('dy/dx',torch.autograd.grad(y,x))

In [0]:
# Convert Between PyTorch Tensor and Numpy Array
cifarImagesTensor = torch.from_numpy(cifarImages)
convertBackNp = cifarImagesTensor.numpy()
print(type(cifarImagesTensor), type(convertBackNp))

# Upload to GPU
cifarImageTensor_GPU = cifarImagesTensor.cuda()
# Download back to CPU
cifarImageTensor_CPU = cifarImageTensor_GPU.cpu()
print('cifarImageTensor_GPU ==>',cifarImageTensor_GPU.device,'cifarImageTensor_CPU ==>', cifarImageTensor_CPU.device)

## Image Classification using Convolutional Neural Network
In this section we will implement the famous LeNet-5 structure using PyTorch and trained on CIFAR-10 dataset.

![alt text](https://cdn-images-1.medium.com/max/2000/1*1TI1aGBZ4dybR6__DI9dzA.png)

You can see the list of PyTorch available layers here => https://pytorch.org/docs/stable/nn.html

### LeNet 5 Network

In [0]:
class LeNet5(nn.Module):
  def __init__(self):
    super(LeNet5,self).__init__()
    self.c1 = nn.Conv2d(3, 6, kernel_size=(5, 5)) 
    self.s2 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)
    self.c3 = nn.Conv2d(6, 16, kernel_size=(5, 5))
    self.s4 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)
    self.c5 = nn.Linear(16*5*5, 120)
    self.f6 = nn.Linear(120, 84)
    self.f7 = nn.Linear(84, 10)
  def forward(self,x, debug=False):

    x = F.relu(self.c1(x))
    x = self.s2(x)
    
    x = F.relu(self.c3(x))
    x = self.s4(x)
    
    # Flatten
    x = x.view(x.size(0),-1)
    
    x = F.relu(self.c5(x))
    
    x = F.relu(self.f6(x))
    x = self.f7(x)
    return x

### Dataset Loader

In [0]:
cifarClassName = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

## Utilize the downloaded dataset ##
class CIFAR10Loader(data.Dataset):
    def __init__(self, cifarDatasetPath='cifar-10-batches-py/',train=True, transform=None):
      self.transform = transform
      
      if train:
        # Train Data
        self.cifarImages = np.empty((0,3072), dtype=np.uint8)
        self.cifarLabels = np.empty((0,), dtype=np.uint8)
        for batchNo in range(1,6):
          dataDict = unpickle(cifarDatasetPath+'/data_batch_'+str(batchNo))
          self.cifarImages = np.vstack((cifarImages, dataDict[b'data']))
          self.cifarLabels = np.hstack((cifarLabels, dataDict[b'labels']))
      
      else:
        # Test Data
        dataDict = unpickle(cifarDatasetPath+'/test_batch')
        self.cifarImages = dataDict[b'data']
        self.cifarLabels = np.array(dataDict[b'labels'])
        
      # Transfrom from (x,3072) ==> (32,32,3,x)
      self.cifarImages = self.cifarImages.reshape(-1,3,32,32).transpose(2,3,1,0)
      
    def __getitem__(self, idx):
        image = self.cifarImages[:,:,:,idx]
        label = self.cifarLabels[idx]
        
        if self.transform:
            image = self.transform(image)
        return image, label

    def __len__(self):
        return self.cifarLabels.shape[0]

###Dataset Transformer 
You can see the list of image augmentation algorithms here ==> https://pytorch.org/docs/stable/torchvision/transforms.html

In [0]:
def transformer(image):
    image = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.406, 0.456), (0.229, 0.225, 0.224)),
        #transforms.RandomHorizontalFlip(p=0.5),
    ])(image)
    return image

In [0]:
cifar10Train = CIFAR10Loader(transform = transformer)
cifar10Test = CIFAR10Loader(train = False, transform = transformer)
cifar10TrainLoader = data.DataLoader(dataset=cifar10Train, batch_size=128,num_workers=4,shuffle=True)
cifar10TestLoader = data.DataLoader(dataset=cifar10Train, batch_size=128,num_workers=4,shuffle=False)

### Create network, optimizer and training loss function

In [0]:
net = LeNet5().cuda()
net.train()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.0001)

### Training

In [0]:
print('== Start Training ==')

bestLoss = float('Inf')
totalEpoch = 20
outputModelFile = 'bestCIFAR10.pth'

for epoch in range(0,totalEpoch):
  totalLoss = 0
  for batchIdx,(image,label) in enumerate(cifar10TrainLoader):
    image, label = image.cuda(), label.cuda()
    
    optimizer.zero_grad()
    
    # forward + backward + optimize
    output = net(image)
    loss = criterion(output, label)

    loss.backward()
    optimizer.step()
    totalLoss+=loss.item()
    
  # Loss decrease, we should save the model
  totalLoss = totalLoss/len(cifar10TrainLoader)
  if totalLoss < bestLoss:
    print('Saving..')
    bestLoss = totalLoss
    state = {
        'model': net.state_dict(),
        'loss': bestLoss,
        'epoch': epoch,
    }
    torch.save(state, outputModelFile)
    
    
  print(epoch+1,'/',str(totalEpoch),'Batch Loss:',totalLoss)
  
print('== Training Finish ==')

### Utilize best model from file 
Perform one shot classification

In [0]:
# Defined new model
bestNet = LeNet5().cuda()
checkpoint = torch.load('bestCIFAR10.pth')
bestNet.load_state_dict(checkpoint['model'])
bestNet.eval()

In [0]:
testIdx = 2
unknownInstance = cifarTestImages[testIdx,:]
unknownInstanceImage = unknownInstance.reshape(3,32,32).transpose(1,2,0)
print('Ground Truth Class:',cifarClassName[cifarTestLabels[testIdx]])
plt.figure(figsize=(2,2))
plt.imshow(unknownInstanceImage)
plt.show()

# The input to network must be 4D Tensor, Eg. you can feed multiple images at once into the network
testImage = transformer(unknownInstanceImage).unsqueeze(0).cuda()
print('testImage Size ==>',testImage.shape)

# Feed forward to network and fetch back result to memory
predicted = F.softmax(bestNet(testImage)).cpu().detach().numpy()[0]
print('Raw Predicted Output', predicted)

# Pick the max value location in array and compare back to class name
predictedClassNo = np.argmax(predicted)
predictedProb = predicted[predictedClassNo]
print('Predicted Class:',cifarClassName[predictedClassNo], '==>',predictedProb)

### Compute model accuracy
This block is your show time. Fill the code to compute the accuracy of your model

In [0]:
### FILL HERE ###

### Assignment 1 : Improve the CIFAR-10 classifier accuracy

From the above section, you can design, train, and test your model on the well-known CIFAR-10 dataset. Your today task is to implement methods to increase the accuracy of CIFAR-10 classifier. Any method can be used and internet is your friend. Nevertheless, you must state the reasons and show the numerical expeimental results on this CIFAR-10 dataset. Don't forget to show your work in step in the below block.

** Do not use test data to train your model! -__-**


Hint : State-of-the-art on CIFAR-10 ==> http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html

In [0]:
## State your improvement here ##


In [0]:
### FILL CODE HERE ###

# Object detection using sliding window based classifier

In this section, we will create a sliding window based car model detection by using the standford car dataset ==> https://www.kaggle.com/jutrera/stanford-car-dataset-by-classes-folder/home

In [0]:
# Download Standford car dataset using kaggle api and 
!kaggle datasets download -d jutrera/stanford-car-dataset-by-classes-folder
!unzip -o stanford-car-dataset-by-classes-folder.zip
!unzip -o -q car_data.zip


In [0]:
# Keep only first 20 classes
keepClass = 20
import shutil

carModelName = []
with open('names.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for idx,row in enumerate(readCSV):
        if idx < keepClass: 
          carModelName.append(row[0])
        else:
          shutil.rmtree('car_data/train/'+row[0].strip().replace('/','-'),ignore_errors=True)
          shutil.rmtree('car_data/test/'+row[0].strip().replace('/','-'),ignore_errors=True)
print(carModelName)

## Dataset Loader and Transformer

In [0]:
# Dataset Directory structure is in the following format : car_data/<set>/<car_model>

transformer = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.405],
                         std=[0.229, 0.224, 0.225]) # Imagenet datset mean and std
])

carTrain = datasets.ImageFolder('car_data/train', transform=transformer)
carTest = datasets.ImageFolder('car_data/test',transform=transformer)
carTrainLoader = data.DataLoader(dataset=carTrain, batch_size=64,num_workers=0,shuffle=True)
carTestLoader = data.DataLoader(dataset=carTest, batch_size=64,num_workers=0,shuffle=False)

## Modify Resnet-18 pretrained model
In this section, we will modify pretrained model on ImageNet dataset to match with the number of our classes (196)

In [0]:
resnet18 = models.resnet18(pretrained=True)
print('=== Before ===')
print(resnet18)
print('=== After ===')
resnet18.fc = nn.Linear(resnet18.fc.in_features,keepClass)
print(resnet18)

## Create network, optimizer and training loss function

In [0]:
resnet18.cuda().train()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(resnet18.parameters(), lr=0.001, weight_decay=0.0001)

## Training

In [0]:
print('== Start Training ==')

bestLoss = float('Inf')
totalEpoch = 20
outputModelFile = 'bestCar.pth'

for epoch in range(0,totalEpoch):
  totalLoss = 0
  trainProgressBar = tqdm(enumerate(carTrainLoader),total=len(carTrainLoader),position=0)
  for batchIdx,(image,label) in trainProgressBar:
    image, label = image.cuda(), label.cuda()
    
    optimizer.zero_grad()
    
    # forward + backward + optimize
    output = resnet18(image)
    loss = criterion(output, label)
    #print(output.shape)
    loss.backward()
    optimizer.step()
    totalLoss+=loss.item()
    avgLoss = totalLoss/(batchIdx+1)
   
    
    trainProgressBar.set_description('avg_loss: %.3f' % avgLoss)
                                                                                                                                   
  # Loss decrease, we should save the model
  totalLoss = totalLoss/len(carTrainLoader)
  if totalLoss < bestLoss:
    print('Saving..')
    bestLoss = totalLoss
    state = {
        'model': resnet18.state_dict(),
        'loss': bestLoss,
        'epoch': epoch,
    }
    torch.save(state, outputModelFile)
    
    
  print(epoch+1,'/',str(totalEpoch),'Epoch Loss:',totalLoss)
  
print('== Training Finish ==')

## Single Shot Classification

In [0]:
# Defined new model
bestCarNet = models.resnet18(pretrained=True)
bestCarNet.fc = nn.Linear(resnet18.fc.in_features,keepClass)
checkpoint = torch.load('bestCar.pth')
print(checkpoint['loss'])
bestCarNet.load_state_dict(checkpoint['model'])
bestCarNet.eval().cuda()

In [0]:
inputImage = cv2.imread('car_data/test/AM General Hummer SUV 2000/00076.jpg')
inputImageRGB = Image.fromarray(cv2.cvtColor(inputImage, cv2.COLOR_BGR2RGB))
inputImageTensor = transformer(inputImageRGB).unsqueeze(0).cuda()

# Feed forward to network and fetch back result to memory
predicted = F.softmax(bestCarNet(inputImageTensor)).cpu().detach().numpy()[0]
print('Raw Predicted Output', predicted)

# Pick the max value location in array and compare back to class name
predictedClassNo = np.argmax(predicted)
print('Predicted Class:',carModelName[predictedClassNo], '==>',predicted[predictedClassNo])

## Compute model accuracy
This block is your show time. Fill the code to compute the accuracy of your model

In [0]:
## FILL HERE ##

## Asssignment 2 : Sliding Window based Detector
In this section, you should be able to write a car model detector by using your trained car model classifier.  The diagram of designed detector would be

Multiscale Inpue Image --> Sliding Window --> Car Model Classifier --> Non-Maxima-Supession (NMS) --> Predicted Car Model and its coordinate

- You can use the entire training set in "car_data/train/" as a training data. 

- Perform an evaluation by using Intersection over Union (IOU) criterion over first k classes (k will be annouced in class) , predicted box will be considered as correct if IOU > 0.5.
- You can use any resorces on internet but you must give a proper credit.
- You can make any modification to trained model to improve overall accuracy.

- ** Please submit result in the following python pickle format and use this guide https://wiki.python.org/moin/UsingPickle to save your work to file. **
        Example:
        output['00001.jpg'] = [x,y,w,h,classNo]
        output['00002.jpg'] = [x,y,w,h,classNo]
        output['00003.jpg'] = [] (No Answer)
       
       
- **(Optional) Top-3 in class accuracy will earn extra points.**

**Don't forget to show your work in step in the below block.**
Expected Algorithm (Image from : https://www.pyimagesearch.com/2015/03/23/sliding-windows-for-object-detection-with-python-and-opencv/ )
![Sliding Window Based Detector](https://www.pyimagesearch.com/wp-content/uploads/2015/03/sliding-window-animated-adrian.gif)

In [0]:
## State your algorithm & improvement here ##


In [0]:
## FILL CODE HERE ##